In [23]:
import sys
import numpy as np
sys.path.insert(0, '../model_functions')
import primary_model as pm
import importlib
importlib.reload(pm)
# importlib.reload()

<module 'primary_model' from '../model_functions/primary_model.py'>

In [2]:
data_path = "../../data/cull%i/model_data/" % 1

devtest_aa_dict = np.load(data_path + 'devtest_aa_dict.npy')[()]
devtest_cmap_dict = np.load(data_path + 'devtest_cmap_dict.npy')[()]
train_aa_dict = np.load(data_path + 'train_aa_dict.npy')[()]
train_cmap_dict = np.load(data_path + 'train_cmap_dict.npy')[()]
valid_aa_dict = np.load(data_path + 'valid_aa_dict.npy')[()]
valid_cmap_dict = np.load(data_path + 'valid_cmap_dict.npy')[()]

In [25]:
model = pm.create_architecture(3, 10)

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, 20)     0                                            
__________________________________________________________________________________________________
1d_convnet_layer1a (Conv1D)     (None, None, 20)     6820        input_layer[0][0]                
__________________________________________________________________________________________________
1d_convnet_batch_norm1a (BatchN (None, None, 20)     80          1d_convnet_layer1a[0][0]         
__________________________________________________________________________________________________
1d_convnet_layer1b (Conv1D)     (None, None, 20)     6820        1d_convnet_batch_norm1a[0][0]    
__________________________________________________________________________________________________
1d_convnet

In [27]:
model.compile(
        optimizer="adam",
#         loss="sparse_categorical_crossentropy",
        loss="binary_crossentropy",
        sample_weight_mode="temporal",
#         metrics=['accuracy']
    )

In [28]:
# history = model.fit_generator(
#         pm.aa_generator(train_aa_dict, train_cmap_dict),
#         validation_data=pm.aa_generator(valid_aa_dict, valid_cmap_dict),
#         steps_per_epoch=5,
#         epochs=2,
#         validation_steps=1,
#         callbacks=pm.callbacks_list
#     )

batch_size = 1
train_steps = round(len(train_aa_dict) / batch_size)
valid_steps = round(len(valid_aa_dict) / batch_size)
print(train_steps)
print(valid_steps)

class_weight = {
        0: 1.,
        1: 8.,
    }

history = model.fit_generator(
    pm.aa_generator_batch(train_aa_dict, train_cmap_dict, batch_size),
    validation_data=pm.aa_generator_batch(
        valid_aa_dict, valid_cmap_dict, batch_size),
    steps_per_epoch=train_steps,
    epochs=20,
    validation_steps=valid_steps,
    callbacks=pm.callbacks_list,
#     class_weight=class_weight,
)

139
40
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
  4/139 [..............................] - ETA: 6:07 - loss: 1.0987

KeyboardInterrupt: 

In [24]:
for x, y, w in pm.aa_generator_batch(train_aa_dict, train_cmap_dict, batch_size=3):
    break

print(x.shape)
print(w.shape)
print(x.shape[1] ** 2)

(3, 350, 20)
(3, 122500)
122500


In [18]:
# y.shape

y.shape[:2]

(3, 53361)

In [ ]:
class_weight = np.zeros((100, 2))
print(class_weight.shape)
class_weight[:, 0] += 1
class_weight[:, 1] += 8
class_weight = class_weight.reshape(-1, 2)

In [29]:
cmap_batch = np.random.randint(2, size=(3,4,5))
class_weight = np.zeros(
    # (aa_batch.shape[0], aa_batch.shape[1] ** 2)
    cmap_batch.shape
)

# class_weight[:, 0] += 1
# class_weight[:, 1] += 8

class_weight += cmap_batch * 8
class_weight[class_weight == 0] = 1

In [30]:
cmap_batch

array([[[0, 0, 0, 0, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 0, 1, 1],
        [0, 1, 1, 0, 0]],

       [[0, 0, 1, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 1, 1]],

       [[0, 0, 1, 0, 0],
        [1, 1, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1]]])

In [31]:
class_weight

array([[[1., 1., 1., 1., 8.],
        [8., 8., 1., 8., 8.],
        [1., 1., 1., 8., 8.],
        [1., 8., 8., 1., 1.]],

       [[1., 1., 8., 8., 1.],
        [8., 1., 1., 8., 1.],
        [8., 8., 1., 8., 1.],
        [1., 1., 1., 8., 8.]],

       [[1., 1., 8., 1., 1.],
        [8., 8., 1., 1., 8.],
        [1., 1., 1., 1., 1.],
        [1., 1., 8., 8., 8.]]])

In [ ]:
class_weight.shape

In [ ]:
# np.zeros((10, 1))
217 * 217

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
test = train_cmap_dict['1a0i']

In [ ]:
# sns.heatmap(test.astype(int))
plt.figure(figsize=(20,10))
plt.imshow(test,)

In [ ]:
x = np.random.random((4,6, 7))
x = x.reshape((*x.shape[:1], -1))
x = np.expand_dims(x, axis=2)

In [ ]:
x.shape

In [33]:
my_log_dir = "Logs/"
!tensorboard --logdir=Logs

W0330 15:21:25.142212 123145518993408 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0330 15:21:25.163197 123145518993408 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0330 15:21:25.214747 123145518993408 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0330 15:21:25.249012 123145518993408 plugin_event_accumulator.py:302] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0330 15:21:25.274074 123145518993408 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or mor

I0330 15:36:13.523386 123145524248576 _internal.py:97] ::1 - - [30/Mar/2019 15:36:13] "GET /data/plugins_listing HTTP/1.1" 200 -
I0330 15:36:44.759546 123145534758912 _internal.py:97] ::1 - - [30/Mar/2019 15:36:44] "GET /data/runs HTTP/1.1" 200 -
I0330 15:36:44.759840 123145529503744 _internal.py:97] ::1 - - [30/Mar/2019 15:36:44] "GET /data/environment HTTP/1.1" 200 -
I0330 15:36:44.761256 123145540014080 _internal.py:97] ::1 - - [30/Mar/2019 15:36:44] "GET /data/experiments HTTP/1.1" 200 -
I0330 15:36:44.763130 123145524248576 _internal.py:97] ::1 - - [30/Mar/2019 15:36:44] "GET /data/plugins_listing HTTP/1.1" 200 -
I0330 15:36:44.773642 123145524248576 _internal.py:97] ::1 - - [30/Mar/2019 15:36:44] "GET /data/plugins_listing HTTP/1.1" 200 -
I0330 15:37:15.654179 123145529503744 _internal.py:97] ::1 - - [30/Mar/2019 15:37:15] "GET /data/runs HTTP/1.1" 200 -
I0330 15:37:15.655136 123145534758912 _internal.py:97] ::1 - - [30/Mar/2019 15:37:15] "GET /data/experiments HTTP/1.1" 200 -
I0

I0330 15:43:24.609749 123145524248576 _internal.py:97] ::1 - - [30/Mar/2019 15:43:24] "GET /data/runs HTTP/1.1" 200 -
I0330 15:43:24.610245 123145540014080 _internal.py:97] ::1 - - [30/Mar/2019 15:43:24] "GET /data/plugins_listing HTTP/1.1" 200 -
I0330 15:43:24.611975 123145529503744 _internal.py:97] ::1 - - [30/Mar/2019 15:43:24] "GET /data/experiments HTTP/1.1" 200 -
I0330 15:43:24.612450 123145534758912 _internal.py:97] ::1 - - [30/Mar/2019 15:43:24] "GET /data/environment HTTP/1.1" 200 -
I0330 15:43:24.622254 123145534758912 _internal.py:97] ::1 - - [30/Mar/2019 15:43:24] "GET /data/plugins_listing HTTP/1.1" 200 -
^C
